In [2]:
import requests
from bs4 import BeautifulSoup
import osc
import os

# Define your search queries
queries = [
    "data privacy corporate compliance",
    "tax reform small business",
    "intellectual property patent law",
    "labor law employee benefits"
]

# Define the folder to save downloaded PDFs
output_folder = "My dataset/bills"
os.makedirs(output_folder, exist_ok=True)

# Define the maximum number of documents to download
max_documents = 1000
downloaded_count = 0

# Base URL for Congress.gov search
base_url = "https://www.congress.gov"

# Function to download PDFs
def download_pdf(pdf_url, folder, bill_name):
    pdf_response = requests.get(pdf_url)
    if pdf_response.status_code == 200:
        with open(os.path.join(folder, f"{bill_name}.pdf"), "wb") as f:
            f.write(pdf_response.content)
        print(f"Downloaded {bill_name}.pdf")
        return True
    else:
        print(f"Failed to download {bill_name}.pdf")
        return False

# Loop through each query
for query in queries:
    if downloaded_count >= max_documents:
        break  # Stop if we've reached the maximum number of documents

    # Loop through Congress 118 and 119
    for congress in [118, 119]:
        if downloaded_count >= max_documents:
            break  # Stop if we've reached the maximum number of documents

        # Construct the search URL with Congress filter
        search_url = f"{base_url}/search?q=%7B%22source%22%3A%22legislation%22%2C%22congress%22%3A%22{congress}%22%2C%22search%22%3A%22{query.replace(' ', '+')}%22%7D"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find links to bill PDFs
        for link in soup.find_all('a', href=True):
            if downloaded_count >= max_documents:
                break  # Stop if we've reached the maximum number of documents

            if "text/pdf" in link['href']:
                pdf_url = base_url + link['href']
                bill_name = link['href'].split("/")[-2]  # Extract bill name from URL
                if download_pdf(pdf_url, output_folder, bill_name):
                    downloaded_count += 1

print(f"Downloaded {downloaded_count} documents.")

Downloaded 0 documents.


In [3]:
import requests
import json

# Set up API Key
API_KEY = "kn9kLejQy6DgoWKrzPNGLouNRTpKcXmeDpDIkfxo"
BASE_URL = "https://api.govinfo.gov"

# Get the list of bills
response = requests.get(f"{BASE_URL}/collections/BILLS/2024?api_key={API_KEY}")
data = response.json()

# Iterate through bills and download PDFs
for item in data["packages"]:
    bill_id = item["packageId"]
    pdf_url = f"{BASE_URL}/packages/{bill_id}/pdf?api_key={API_KEY}"
    
    pdf_response = requests.get(pdf_url)
    with open(f"{bill_id}.pdf", "wb") as f:
        f.write(pdf_response.content)

print("All bill PDFs downloaded successfully!")


KeyError: 'packages'

In [4]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.congress.gov"
SEARCH_URL = BASE_URL + "/search?q={%22source%22:%22legislation%22}&searchResultViewType=expanded"

# Get list of bills
response = requests.get(SEARCH_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Find bill links
bill_links = soup.find_all("a", class_="result-heading")

for link in bill_links[:5]:  # Download first 5 bills as an example
    bill_page = requests.get(BASE_URL + link["href"])
    bill_soup = BeautifulSoup(bill_page.text, "html.parser")
    
    # Find the PDF link
    pdf_link = bill_soup.find("a", string="PDF")
    if pdf_link:
        pdf_url = BASE_URL + pdf_link["href"]
        pdf_response = requests.get(pdf_url)
        
        # Save PDF
        bill_title = link.text.strip().replace(" ", "_") + ".pdf"
        with open(bill_title, "wb") as f:
            f.write(pdf_response.content)

print("Bill PDFs downloaded successfully!")

Bill PDFs downloaded successfully!


In [18]:
import requests

API_KEY = "o0EmHjN454oYBCzhdyPMNwinc0MnfAnjYRbWVCRu"  # Replace with your actual API key
BASE_URL = "https://api.govinfo.gov/packages"

def search_bills(query, congress):
    """Search GovInfo API for bills matching the query."""
    url = f"{BASE_URL}/BILLS-{congress}-search.json"
    params = {
        "query": query,
        "api_key": API_KEY
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        bills = response.json().get("packages", [])
        return [(bill["packageId"], bill["title"]) for bill in bills]
    
    print(f"⚠️ API Error {response.status_code} for query: {query}")
    return []

# Run search for both Congress sessions
congress_sessions = ["118", "119"]
queries = [
    "privacy"
    # "data privacy",
    # "tax reform",
    # "intellectual property",
    # "labor law"
]

for congress in congress_sessions:
    for query in queries:
        print(f"\n🔍 Searching for '{query}' in {congress}th Congress...")
        results = search_bills(query, congress)
        
        if not results:
            print("⚠️ No matching bills found!")
        else:
            for bill_id, title in results[:5]:  # Show first 5 results
                print(f"📄 Bill: {title} (ID: {bill_id})")


🔍 Searching for 'privacy' in 118th Congress...
⚠️ API Error 500 for query: privacy
⚠️ No matching bills found!

🔍 Searching for 'privacy' in 119th Congress...
⚠️ API Error 500 for query: privacy
⚠️ No matching bills found!


In [12]:
import requests

API_KEY = "C5UjVSoSkc3FHGWS194refWR87p3etqc6vHX0C9E"
TEST_URL = f"https://api.govinfo.gov/packages/BILLS-118hr5ih/summary?api_key={API_KEY}"

response = requests.get(TEST_URL)
print(response.status_code, response.text)  # Should be 200 if key is valid


200 {
    "originChamber": "HOUSE",
    "references": [{
        "contents": [{
            "label": "U.S.C",
            "title": "20",
            "sections": [
                "1232g",
                "1232h",
                "1400",
                "6311",
                "6312",
                "7801"
            ]
        }],
        "collectionCode": "USCODE",
        "collectionName": "U.S. Code"
    }],
    "congress": "118",
    "session": "1",
    "detailsLink": "https://www.govinfo.gov/app/details/BILLS-118hr5ih",
    "shortTitle": [{
        "type": "measure",
        "title": "Parents Bill of Rights Act"
    }],
    "isPrivate": "false",
    "title": "To ensure the rights of parents are honored and protected in the Nation\u2019s public schools.",
    "branch": "legislative",
    "isAppropriation": "false",
    "collectionName": "Congressional Bills",
    "download": {
        "premisLink": "https://api.govinfo.gov/packages/BILLS-118hr5ih/premis",
        "xmlLink": "https

In [27]:
import requests

API_KEY = "o0EmHjN454oYBCzhdyPMNwinc0MnfAnjYRbWVCRu"
BASE_URL = "https://api.congress.gov/v3/bill"

congress = 118  # Use a valid congress number (118 or 119)
bill_type = "hr"  # "hr" for House Bills or "s" for Senate Bills

# Set headers to avoid 403 errors
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to fetch all bills
def fetch_all_bills(congress, bill_type):
    all_bills = []
    page = 1
    while True:
        params = {
            "api_key": API_KEY,
            "page": page  # Pagination
        }

        # Build the API URL
        url = f"{BASE_URL}/{congress}/{bill_type}"
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            print(f"Response for page {page}: {response.text[:500]}")  # Print first 500 characters of the response for inspection
            data = response.json()

            # Check if 'bills' key is present
            if "bills" in data:
                bills = data["bills"]
                all_bills.extend(bills)
                print(f"Fetched {len(bills)} bills from page {page}")

                # If there's no 'next' key or it indicates the last page, break the loop
                if "next" in data and data["next"]:
                    page += 1
                else:
                    break
            else:
                print("No bills found in response.")
                break
        else:
            print(f"Error: {response.status_code} for page {page}")
            break

    return all_bills

# Fetch all bills
bills = fetch_all_bills(congress, bill_type)
print(f"Total bills retrieved: {len(bills)}")

# Optionally print first few bills for verification
for bill in bills[:5]:
    print(f"Bill Title: {bill.get('title')}")
    print(f"Bill Number: {bill.get('number')}")
    print("="*50)


Response for page 1: {
    "bills": [
        {
            "congress": 118,
            "latestAction": {
                "actionDate": "2025-01-06",
                "text": "Became Public Law No: 118-274."
            },
            "number": "4984",
            "originChamber": "House",
            "originChamberCode": "H",
            "title": "D.C. Robert F. Kennedy Memorial Stadium Campus Revitalization Act",
            "type": "HR",
            "updateDate": "2025-03-21",
            "updateDateIncludingText
Fetched 20 bills from page 1
Total bills retrieved: 20
Bill Title: D.C. Robert F. Kennedy Memorial Stadium Campus Revitalization Act
Bill Number: 4984
Bill Title: Social Security Fairness Act of 2023
Bill Number: 82
Bill Title: To designate the facility of the United States Postal Service located at 119 North Anderson Street in Elwood, Indiana, as the "Officer Noah Jacob Shahnavaz Post Office Building".
Bill Number: 9775
Bill Title: To designate the facility of the United S

In [30]:
import requests

API_KEY = "o0EmHjN454oYBCzhdyPMNwinc0MnfAnjYRbWVCRu"
BASE_URL = "https://api.congress.gov/v3/bill"

congress = 118  # Use the 118th Congress
bill_type = "hr"  # "hr" for House Bills, "s" for Senate Bills

# Set headers to avoid 403 errors
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to fetch bills until 1000 bills are retrieved
def fetch_bills_up_to_limit(congress, bill_type, target_count=1000):
    all_bills = []
    page = 1
    while len(all_bills) < target_count:
        params = {
            "api_key": API_KEY,
            "page": page  # Pagination
        }

        # Build the API URL
        url = f"{BASE_URL}/{congress}/{bill_type}"
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            data = response.json()

            # Check if 'bills' key is present
            if "bills" in data:
                bills = data["bills"]
                all_bills.extend(bills)
                print(f"Fetched {len(bills)} bills from page {page}, Total bills: {len(all_bills)}")

                # Check if there are more bills to fetch
                if "next" in data and data["next"] is not None:
                    page += 1  # Move to the next page
                else:
                    print("No more pages to fetch.")
                    break
            else:
                print("No bills found in response.")
                break
        else:
            print(f"Error: {response.status_code} for page {page}")
            break

    return all_bills

# Fetch bills up to 1000 from the 118th Congress
bills = fetch_bills_up_to_limit(congress, bill_type, target_count=1000)
print(f"Total bills retrieved: {len(bills)}")

# Optionally print first few bills for verification
for bill in bills[:5]:
    print(f"Bill Title: {bill.get('title')}")
    print(f"Bill Number: {bill.get('number')}")
    print("="*50)

Fetched 20 bills from page 1, Total bills: 20
No more pages to fetch.
Total bills retrieved: 20
Bill Title: D.C. Robert F. Kennedy Memorial Stadium Campus Revitalization Act
Bill Number: 4984
Bill Title: Social Security Fairness Act of 2023
Bill Number: 82
Bill Title: To designate the facility of the United States Postal Service located at 119 North Anderson Street in Elwood, Indiana, as the "Officer Noah Jacob Shahnavaz Post Office Building".
Bill Number: 9775
Bill Title: To designate the facility of the United States Postal Service located at 119 Main Street in Plains, Georgia, as the "Jimmy and Rosalynn Carter Post Office".
Bill Number: 9600
Bill Title: Federal Register Modernization Act of 2024
Bill Number: 9592
